In [69]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy import stats

etfs = ['SPY', 'IDTL.L', '^IXIC']
# etfs = ['SPY']

returns = yf.download(etfs, end='2024-08-31')['Adj Close'].pct_change(fill_method = None).dropna()

[*********************100%%**********************]  3 of 3 completed


In [70]:
returns.head()

Ticker,IDTL.L,SPY,^IXIC
Date,,,
2015-01-22,0.000000,0.014870,0.017779
2015-01-23,0.001996,-0.005483,0.001575
2015-01-26,0.000000,0.002342,0.002917
2015-01-27,0.009960,-0.013191,-0.018915
2015-01-28,-0.000740,-0.012825,-0.009294


In [71]:
# Calculate cumulative return
cumulative_return = (1 + returns).prod() - 1

# Number of trading days in a year
trading_days = 252

# Calculate annualized return
annualized_return = (1 + cumulative_return) ** (trading_days / len(returns)) - 1



In [72]:
weights = pd.Series({
    'SPY': 0.05,
    'IDTL': 0.9,
    '^IXIC': 0.05,
})


# Calculate the weighted average return
weighted_average_return = (annualized_return * weights).sum()

print(f'Weighted Average Return: {weighted_average_return:.2%}')

Weighted Average Return: 1.40%


In [73]:
# Calculate the covariance matrix
cov_matrix = returns.cov()

# Calculate the portfolio standard deviation
std= np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
annual_std = std * np.sqrt(trading_days)

print(f'Portfolio Annualized Standard Deviation: {annual_std:.4f}')


Portfolio Annualized Standard Deviation: 0.1706


In [74]:
# Number of bootstrap samples
n_samples = 100000

# Simulate portfolio returns by randomly sampling from the historical data
bootstrap_returns = np.random.choice(returns.mean(axis=1) * trading_days, n_samples, replace=True)

# Calculate the probability of returns over the target return
prob_over_target = np.mean(bootstrap_returns > 0.06)

print(f'Probability that the portfolio will have returns over 6% (bootstrap): {prob_over_target:.2%}')




Probability that the portfolio will have returns over 6% (bootstrap): 53.22%
